In [1]:
# load in relevant files
import pandas as pd
market_df = pd.read_csv('ygo_card_data_202408300224.csv')
cards_df = pd.read_csv('202408041907.csv')
sales_df = pd.read_csv('ygo_sales_data_202408311524.csv')

In [3]:
market_df

,Unnamed: 0,name,number,rarity,price,quantity,quantity_sellers,price_change_3m,index,asof
0,0,Gouki Guts - 2021 Tin of Ancient Battles (MP21),MP21-EN046,Common,$0.05,2314,212,(-14.29%),249048,1.723611e+09
1,1,Branded Disciple - Battle of Chaos (BACH),BACH-EN053,Common,$0.08,Market,Market,(+14.29%),262318,1.723601e+09
2,2,Chimeratech Overdragon - Star Pack 2014,SP14-EN043,Common,-,Market,Market,NaN,79857,1.723916e+09
3,3,Caius the Shadow Monarch - Gold Series 2009 (G...,GLD2-EN033,Ultra Rare,$4.24,Market,Market,(-5.05%),32106,1.724124e+09
4,4,De-Fusion - Duelist Pack 4: Zane Truesdale (DP04),DP04-EN017,Common,$0.23,335,80,NaN,26515,1.724172e+09
...,...,...,...,...,...,...,...,...,...,...
39894,39894,Pazuzule - 2022 Tin of the Pharaoh's Gods (MP22),MP22-EN173,Common,$0.06,3583,253,(+25.00%),285022,1.723572e+09
39895,39895,Steelswarm Sting - Duel Terminal 6 (DT06),DT06-EN030,Duel Terminal Rare Parallel Rare,-,217,26,NaN,81461,1.723863e+09
39896,39896,Protector with Eyes of Blue - Legendary Decks ...,LDK2-ENK07,Common,$0.13,Market,Market,(+30.00%),543085,1.723444e+09
39897,39897,One for One - Structure Deck: Pendulum Dominat...,SDPD-EN028,Common,$0.55,362,67,(-16.92%),127050,1.723787e+09


In [5]:
cards_df

,Unnamed: 0,index,name,description,type,sub_type,attribute,rank,attack,defense,set_name,set_id,set_release,rarity
0,0,7128,"""A"" Cell Breeding Device","During each of your Standby Phases, put 1 A-Co...",SPELL,Continuous,NaN,NaN,NaN,NaN,FORCE OF THE BREAKER,FOTB-EN043,2007-05-16,C Common
1,1,7315,"""A"" Cell Incubator",Each time an A-Counter(s) is removed from play...,SPELL,Continuous,NaN,NaN,NaN,NaN,GLADIATOR'S ASSAULT,GLAS-EN062,2007-11-14,C Common
2,2,12653,"""A"" Cell Recombination Device",Target 1 face-up monster on the field; send 1 ...,SPELL,Quick-Play,NaN,NaN,NaN,NaN,INVASION: VENGEANCE,INOV-EN063,2016-11-04,C Common
3,3,6994,"""A"" Cell Scatter Burst","Select 1 face-up ""Alien"" monster you control. ...",SPELL,Quick-Play,NaN,NaN,NaN,NaN,STRIKE OF NEOS,STON-EN041,2007-02-28,C Common
4,4,18843,"""Infernoble Arms - Almace""",While this card is equipped to a monster: You ...,SPELL,Equip,NaN,NaN,NaN,NaN,DUELIST NEXUS,DUNE-EN056,2023-07-28,UR Ultra Rare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36208,36208,10366,ZW - Ultimate Shield,When this card is Normal or Special Summoned: ...,MONSTER,[Aqua/Effect],EARTH,Level 4,0,2000,COSMO BLAZER,CBLZ-EN007,2013-01-25,C Common
36209,36209,9879,ZW - Unicorn Spear,"You can target 1 ""Number C39: Utopia Ray"" you ...",MONSTER,[Beast/Effect],LIGHT,Level 4,1900,0,STAR PACK 2014,SP14-EN004,2014-02-21,C Common
36210,36210,9879,ZW - Unicorn Spear,"You can target 1 ""Number C39: Utopia Ray"" you ...",MONSTER,[Beast/Effect],LIGHT,Level 4,1900,0,STAR PACK 2014,SP14-EN004,2014-02-21,ST Starfoil
36211,36211,9879,ZW - Unicorn Spear,"You can target 1 ""Number C39: Utopia Ray"" you ...",MONSTER,[Beast/Effect],LIGHT,Level 4,1900,0,SUPER STARTER V FOR VICTORY,YS13-EN018,2013-06-14,C Common


In [93]:
sales_df

,Unnamed: 0,id,valid_time,condition,variant,language,quantity,title,listingType,customListingId,purchasePrice,shippingPrice,orderDate,index
0,0,22100,1.724268e+09,Lightly Played,Unlimited,English,1.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.25,1.27,2024-08-18T20:14:42.113+00:00,22100
1,1,22100,1.724268e+09,Moderately Played,Unlimited,English,1.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.04,0.95,2024-08-13T21:02:28.257+00:00,22100
2,2,22100,1.724268e+09,Near Mint,Unlimited,English,5.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.11,0.00,2024-08-13T00:56:53.23+00:00,22100
3,3,22100,1.724268e+09,Lightly Played,Unlimited,English,5.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.09,0.00,2024-08-13T00:56:53.23+00:00,22100
4,4,22100,1.724268e+09,Near Mint,1st Edition,English,1.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,1.24,0.00,2024-08-11T15:17:49.81+00:00,22100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703505,703505,116961,1.723815e+09,Lightly Played,1st Edition,English,1.0,Rebellion,ListingWithoutPhotos,0,0.33,1.22,2024-05-27T22:44:47.483+00:00,116961
703506,703506,116961,1.723815e+09,Moderately Played,1st Edition,English,1.0,Rebellion,ListingWithoutPhotos,0,0.29,1.20,2024-05-24T03:15:32.61+00:00,116961
703507,703507,116961,1.723815e+09,Near Mint,1st Edition,English,1.0,Rebellion,ListingWithoutPhotos,0,0.52,0.00,2024-05-23T15:57:59.71+00:00,116961
703508,703508,116961,1.723815e+09,Near Mint,1st Edition,English,1.0,Rebellion,ListingWithoutPhotos,0,0.33,1.22,2024-05-23T02:20:17.2+00:00,116961


In [7]:
rare_lookup = {'Rare': 'R',
 'Gold Rare': 'GR',
 'Starfoil Rare': 'ST',
 '10000 Secret Rare': '10000 SE',
 'Platinum Secret Rare': 'PS',
 'Super Rare': 'SR',
 'Common': 'C',
 'Quarter Century Secret Rare': 'QCSE',
 'Shatterfoil Rare': 'SH',
 'Secret Rare': 'SE',
 'Prismatic Ultimate Rare': 'PSE',
 'Secret Pharaoh’s Rare': 'SE (PR)',
 'Ultra Rare': 'UR',
 'Premium Gold Rare': 'PGR',
 'Promo': None,
 'Ultra Pharaoh’s Rare': 'UR (PR)',
 "Prismatic Collector's Rare": 'PSE',
 'Platinum Rare': 'PL',
 'Ghost Rare': 'GR',
 'Parallel Rare': None,
 'Starlight Rare': 'STAR',
 'Ultimate Rare': 'UL',
 'Ghost/Gold Rare': 'GH',
 "Collector's Rare": 'CR',
 'Prismatic Secret Rare': 'PSE',
 'Mosaic Rare': 'MR',
 'Gold Secret Rare': 'GSE'
}

In [9]:
rarity_konami = {'10000 SE 10000 SECRET RARE': '10000 SE',
 'C Common': 'C',
 "CR COLLECTOR'S RARE": 'CR',
 'GH Ghost Rare': 'GH',
 'GR Gold Rare': 'GR',
 'GSE Gold Secret': 'GSE',
 'H Hobby': 'H',
 'MR Mosaic Rare': 'MR',
 'PGR Premium Gold Rare': 'PGR',
 'PL Platinum Rare': 'PL',
 'PS Platinum Secret Rare': 'PS',
 'PSE Prismatic Secret Rare': 'PSE',
 'QCSE Quarter Century Secret Rare': 'QCSE',
 'R Rare': 'R',
 'SE Secret Rare': 'SE',
 'SH Shattefoil': 'SH',
 'SR Super Rare': 'SR',
 'ST Starfoil': 'ST',
 'STAR Starlight Rare': 'STAR',
 'UL Ultimate Rare': 'UL',
 'UR (Hobby) Ultra Rare（Hobby League Version）': 'UR (Hobby)',
 "UR (PR) Ultra Rare (Pharaoh's Rare)": 'UR (PR)',
 'UR Ultra Rare': 'UR',
 'UR Ultra Rare (Duelist Saga Version)': 'UR (Duelist Saga Version)'
}

In [11]:
[rarity for rarity in set(market_df['rarity']) if not pd.isnull(rarity) and 'Duel Terminal' not in rarity]

['Ultra Rare',
 'Mosaic Rare',
 'Secret Rare',
 'Ultimate Rare',
 'Prismatic Ultimate Rare',
 'Gold Rare',
 'Ghost/Gold Rare',
 'Starlight Rare',
 'Common',
 'Quarter Century Secret Rare',
 'Platinum Rare',
 'Parallel Rare',
 'Premium Gold Rare',
 'Secret Pharaoh’s Rare',
 'Ultra Pharaoh’s Rare',
 "Collector's Rare",
 'Platinum Secret Rare',
 'Rare',
 'Prismatic Secret Rare',
 'Super Rare',
 'Promo',
 '10000 Secret Rare',
 'Shatterfoil Rare',
 "Prismatic Collector's Rare",
 'Ghost Rare',
 'Gold Secret Rare',
 'Starfoil Rare']

In [13]:
set(cards_df['rarity'])

{'10000 SE 10000 SECRET RARE',
 'C Common',
 "CR COLLECTOR'S RARE",
 'GH Ghost Rare',
 'GR Gold Rare',
 'GSE Gold Secret',
 'H Hobby',
 'MR Mosaic Rare',
 'PGR Premium Gold Rare',
 'PL Platinum Rare',
 'PS Platinum Secret Rare',
 'PSE Prismatic Secret Rare',
 'QCSE Quarter Century Secret Rare',
 'R Rare',
 'SE Secret Rare',
 'SH Shattefoil',
 'SR Super Rare',
 'ST Starfoil',
 'STAR Starlight Rare',
 'UL Ultimate Rare',
 'UR (Hobby) Ultra Rare（Hobby League Version）',
 "UR (PR) Ultra Rare (Pharaoh's Rare)",
 'UR Ultra Rare',
 'UR Ultra Rare (Duelist Saga Version)'}

In [15]:
cards_df['join_id'] = cards_df.apply(lambda card: f"{card['set_id']}{rarity_konami.get(card['rarity'])}", axis=1)

In [17]:
market_df['join_id'] = market_df.apply(lambda card: f"{card['number']}{rare_lookup.get(card['rarity'])}", axis=1)

In [33]:
merged_df = cards_df.merge(market_df[['name', 'join_id', 'price', 'price_change_3m', 'index', 'asof']],
                           on='join_id',
                           suffixes=[None, '_market']
                          )

In [37]:
len(merged_df)

36527

In [35]:
merged_df[merged_df['name'] == 'Purrely']

,Unnamed: 0,index,name,description,type,sub_type,attribute,rank,attack,defense,set_name,set_id,set_release,rarity,join_id,name_market,price,price_change_3m,index_market,asof
25365,25219,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy/Effect],LIGHT,Level 1,100,100,25TH ANNIVERSARY RARITY COLLECTION 2,RA02-EN018,2024-05-24,SR Super Rare,RA02-EN018SR,Purrely - 25th Anniversary Rarity Collection I...,$0.19,(-91.91%),550968,1.723438e+09
25366,25220,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy/Effect],LIGHT,Level 1,100,100,25TH ANNIVERSARY RARITY COLLECTION 2,RA02-EN018,2024-05-24,UR Ultra Rare,RA02-EN018UR,Purrely (UR) - 25th Anniversary Rarity Collect...,$0.16,(-93.13%),550969,1.723438e+09
25367,25221,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy/Effect],LIGHT,Level 1,100,100,25TH ANNIVERSARY RARITY COLLECTION 2,RA02-EN018,2024-05-24,SE Secret Rare,RA02-EN018SE,Purrely (Secret Rare) - 25th Anniversary Rarit...,$0.52,(-87.61%),550970,1.723438e+09
25368,25223,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy/Effect],LIGHT,Level 1,100,100,25TH ANNIVERSARY RARITY COLLECTION 2,RA02-EN018,2024-05-24,PS Platinum Secret Rare,RA02-EN018PS,Purrely (Platinum Secret Rare) - 25th Anniver...,$2.25,(-70.91%),550971,1.723438e+09
25369,25225,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy/Effect],LIGHT,Level 1,100,100,25TH ANNIVERSARY RARITY COLLECTION 2,RA02-EN018,2024-05-24,QCSE Quarter Century Secret Rare,RA02-EN018QCSE,Purrely (Quarter Century Secret Rare) - 25th A...,$9.72,(-75.69%),550972,1.723438e+09
25370,25226,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy/Effect],LIGHT,Level 1,100,100,AMAZING DEFENDERS,AMDE-EN013,2023-01-20,UR Ultra Rare,AMDE-EN013UR,Purrely - Amazing Defenders (AMDE),$7.11,(-31.00%),477702,1.723552e+09
25371,25227,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy/Effect],LIGHT,Level 1,100,100,AMAZING DEFENDERS,AMDE-EN013,2023-01-20,CR COLLECTOR'S RARE,AMDE-EN013CR,Purrely (CR) - Amazing Defenders (AMDE),$22.46,(-25.78%),477703,1.723552e+09


In [47]:
import dateutil
import datetime

def average_time_sale(times, format='unix'):
    '''
    Given a list of time's, we will calculate the 
    average difference between those times

    Parameters
    ----------
    times list
        A list of the times where each element is a timestamp.
        All times must be in the same format

    format (optional) String
        The format of the times in the `times` list. By default,
        it will be from the 'unix' value. Currently, all other 
        values will use dateutil.parser.parse()

    References
    ----------
    https://docs.python.org/3/library/datetime.html#timedelta-objects
    '''
    # if times is empty or has only 1, we can't calculate the average
    if len(times) < 2:
        return False
    # malformed values
    for ts in times:
        if pd.isnull(ts):
            return False
    
    # format the times to Python's datetime data structure
    if format == 'unix':
        formatted_times = [datetime.fromtimestamp(float(ts)) for ts in times]
        formatted_times.sort()
    else :
        formatted_times = [dateutil.parser.parse(str(ts)) for ts in times]
        formatted_times.sort()
    
    # create time delta's
    time_deltas = [formatted_times[i] - formatted_times[i - 1] for i in range(1, len(formatted_times))]

    # calculate average using arithmetic mean
    average = sum([delta.total_seconds() for delta in time_deltas])/len(time_deltas)

    # units are in days, average is calculated at seconds
    result = (average / 60) / 24 # secs -> hrs -> days
    return result

In [49]:
# the ID column is the index for tcg player, needs to be renamed
sales_df['index'] = sales_df['id']
merged_sales = sales_df.merge(market_df[['index', 'join_id']], on='index')

In [51]:
merged_sales['avg_sale_days'] = merged_sales.groupby('join_id')['orderDate'].transform(lambda x: average_time_sale(x, format='iso'))

In [52]:
merged_sales

,Unnamed: 0,id,valid_time,condition,variant,language,quantity,title,listingType,customListingId,purchasePrice,shippingPrice,orderDate,index,join_id,avg_sale_days
0,0,22100,1.724268e+09,Lightly Played,Unlimited,English,1.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.25,1.27,2024-08-18T20:14:42.113+00:00,22100,MRD-101C,338.863303
1,1,22100,1.724268e+09,Moderately Played,Unlimited,English,1.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.04,0.95,2024-08-13T21:02:28.257+00:00,22100,MRD-101C,338.863303
2,2,22100,1.724268e+09,Near Mint,Unlimited,English,5.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.11,0.00,2024-08-13T00:56:53.23+00:00,22100,MRD-101C,338.863303
3,3,22100,1.724268e+09,Lightly Played,Unlimited,English,5.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.09,0.00,2024-08-13T00:56:53.23+00:00,22100,MRD-101C,338.863303
4,4,22100,1.724268e+09,Near Mint,1st Edition,English,1.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,1.24,0.00,2024-08-11T15:17:49.81+00:00,22100,MRD-101C,338.863303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703505,703505,116961,1.723815e+09,Lightly Played,1st Edition,English,1.0,Rebellion,ListingWithoutPhotos,0,0.33,1.22,2024-05-27T22:44:47.483+00:00,116961,MIL1-EN013UR,208.927145
703506,703506,116961,1.723815e+09,Moderately Played,1st Edition,English,1.0,Rebellion,ListingWithoutPhotos,0,0.29,1.20,2024-05-24T03:15:32.61+00:00,116961,MIL1-EN013UR,208.927145
703507,703507,116961,1.723815e+09,Near Mint,1st Edition,English,1.0,Rebellion,ListingWithoutPhotos,0,0.52,0.00,2024-05-23T15:57:59.71+00:00,116961,MIL1-EN013UR,208.927145
703508,703508,116961,1.723815e+09,Near Mint,1st Edition,English,1.0,Rebellion,ListingWithoutPhotos,0,0.33,1.22,2024-05-23T02:20:17.2+00:00,116961,MIL1-EN013UR,208.927145


In [73]:
output_df = merged_df.merge(merged_sales[['join_id', 'avg_sale_days']].drop_duplicates(), on='join_id', how='left')
output_df

,Unnamed: 0,index,name,description,type,sub_type,attribute,rank,attack,defense,...,set_id,set_release,rarity,join_id,name_market,price,price_change_3m,index_market,asof,avg_sale_days
0,0,7128,"""A"" Cell Breeding Device","During each of your Standby Phases, put 1 A-Co...",SPELL,Continuous,NaN,NaN,NaN,NaN,...,FOTB-EN043,2007-05-16,C Common,FOTB-EN043C,"""A"" Cell Breeding Device - Force of the Breake...",$0.22,(+5.56%),25767,1.724181e+09,303.975517
1,1,7315,"""A"" Cell Incubator",Each time an A-Counter(s) is removed from play...,SPELL,Continuous,NaN,NaN,NaN,NaN,...,GLAS-EN062,2007-11-14,C Common,GLAS-EN062C,"""A"" Cell Incubator - Gladiator's Assault (GLAS)",$0.21,(-4.17%),26309,1.724175e+09,531.957203
2,2,12653,"""A"" Cell Recombination Device",Target 1 face-up monster on the field; send 1 ...,SPELL,Quick-Play,NaN,NaN,NaN,NaN,...,INOV-EN063,2016-11-04,C Common,INOV-EN063C,"""A"" Cell Recombination Device - Invasion: Veng...",$0.25,(+4.55%),124766,1.723790e+09,198.589659
3,3,6994,"""A"" Cell Scatter Burst","Select 1 face-up ""Alien"" monster you control. ...",SPELL,Quick-Play,NaN,NaN,NaN,NaN,...,STON-EN041,2007-02-28,C Common,STON-EN041C,"""A"" Cell Scatter Burst - Strike of Neos (STON)",$0.11,(-14.29%),25692,1.724182e+09,328.55358
4,4,18843,"""Infernoble Arms - Almace""",While this card is equipped to a monster: You ...,SPELL,Equip,NaN,NaN,NaN,NaN,...,DUNE-EN056,2023-07-28,UR Ultra Rare,DUNE-EN056UR,"""Infernoble Arms - Almace"" - Duelist Nexus (DUNE)",$0.37,(-37.29%),504871,1.723500e+09,30.220306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36522,36208,10366,ZW - Ultimate Shield,When this card is Normal or Special Summoned: ...,MONSTER,[Aqua/Effect],EARTH,Level 4,0,2000,...,CBLZ-EN007,2013-01-25,C Common,CBLZ-EN007C,ZW - Ultimate Shield - Cosmo Blazer (CBLZ),$0.01,NaN,67602,1.723960e+09,940.630348
36523,36209,9879,ZW - Unicorn Spear,"You can target 1 ""Number C39: Utopia Ray"" you ...",MONSTER,[Beast/Effect],LIGHT,Level 4,1900,0,...,SP14-EN004,2014-02-21,C Common,SP14-EN004C,ZW - Unicorn Spear - Star Pack 2014,-,NaN,79818,1.723917e+09,5556.628662
36524,36210,9879,ZW - Unicorn Spear,"You can target 1 ""Number C39: Utopia Ray"" you ...",MONSTER,[Beast/Effect],LIGHT,Level 4,1900,0,...,SP14-EN004,2014-02-21,ST Starfoil,SP14-EN004ST,ZW - Unicorn Spear (Starfoil) - Star Pack 2014,$0.87,(-1.12%),79819,1.723917e+09,2049.18244
36525,36211,9879,ZW - Unicorn Spear,"You can target 1 ""Number C39: Utopia Ray"" you ...",MONSTER,[Beast/Effect],LIGHT,Level 4,1900,0,...,YS13-EN018,2013-06-14,C Common,YS13-EN018C,ZW - Unicorn Spear - Super Starter: V for Victory,$0.14,NaN,69047,1.723955e+09,626.891378


In [89]:
output_df['price_asof'] = output_df['asof'].apply(lambda asof: datetime.datetime.fromtimestamp(asof).strftime('%B %d, %Y'))
output_df['market_name'] = output_df['name_market']
output_df['price_change'] = output_df['price_change_3m']

In [98]:
output_df[['name', 'description', 'type', 'sub_type', 'attribute', 'rank', 'attack', 'defense',
 'rarity', 'set_name', 'set_id', 'set_release',
 'market_name', 'price', 'price_asof', 'price_change', 'avg_sale_days',
 'index', 'index_market', 'join_id']].to_csv('202408312230.csv', index_label=False)